In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array
mat = np.mat

# 读取训练集

In [19]:
dirPath = "data preprocessed\\unnormalized"

trainSet_origin = np.loadtxt(dirPath + '\\train.csv', delimiter=",")
validateSet_origin = np.loadtxt(dirPath + '\\validate.csv', delimiter=",")
testSet = np.loadtxt(dirPath + '\\test.csv', delimiter=",")

trainSet_origin.shape, validateSet_origin.shape, testSet.shape

((33600, 18), (14400, 18), (12000, 17))

In [20]:
trainSet_merge = np.row_stack([trainSet_origin, validateSet_origin])
trainSet_merge.shape

(48000, 18)

In [21]:
trainSet, trainSetLabel = trainSet_merge[:, :-1], trainSet_merge[:, -1]
trainSet.shape

(48000, 17)

# 实现PCA

In [22]:
def PCA(dataMat, topNfeat=9999999): 
    '''
    输入：数据集，主成分数目
    输出：包含数据的列表，reconmat
    描述：降维后的数据集，保留的主成分所占的方差百分比
    ''' 
    meanVals = np.mean(dataMat, axis=0) #计算数据的均值，axis=0代表按列求均值
    meanRemoved = dataMat - meanVals    #将数据减去其原来的均值
    covMat = np.cov(meanRemoved, rowvar=0)     #得到协方差矩阵，rowvar=0代表传入的数据一行代表一个样本
                                               #（rowvar=1则代表传入的数据一列代表一个样本）
    eigVals, eigVects = np.linalg.eig(mat(covMat)) #得到特征值和特征向量，两者一一对应
    eigValIndex = np.argsort(eigVals)              #对特征值从小到大排列,得到排列后的下标
                                                 #例：eigVals=[2.5,7.0,0.5],
                                                 #则  eigValIndex=[1,2,0]
    eigValIndex = eigValIndex[:-(topNfeat+1):-1] #逆序得到topNfeat个最大的特征值
    redEigVects = eigVects[:,eigValIndex]        #得到topNfeat个最大的特征向量
    lowDDataMat = meanRemoved * redEigVects      #将数据降到topNfeat维
    varRate = np.sum(eigVals[eigValIndex])/np.sum(eigVals)*100 #得到保留的主成分的方差百分比
    return lowDDataMat, varRate

# 测试主成分个数

In [23]:
totFeatureNum = trainSet.shape[1]
preVarRate=0.0
print("主成分        方差百分比      累积方差百分比")
for i in tnrange(1, totFeatureNum+1):
        lowDDataMat, tolVarRate = PCA(mat(trainSet), i)           #主成分数目为i时的累积方差百分比
        curVarRate = tolVarRate-preVarRate #当前新增加的主成分所占的方差百分比
        print("%2d               %5.2f             %5.2f"%(i, curVarRate, tolVarRate))
        preVarRate = tolVarRate

主成分        方差百分比      累积方差百分比
 1               59.28             59.28
 2               22.46             81.74
 3                8.09             89.83
 4                6.35             96.18
 5                1.52             97.70
 6                1.08             98.78
 7                1.01             99.80
 8                0.18             99.97
 9                0.03             100.00
10                0.00             100.00
11                0.00             100.00
12                0.00             100.00
13                0.00             100.00
14                0.00             100.00
15                0.00             100.00
16                0.00             100.00
17                0.00             100.00



这里发现，前9个成分保存了100%的信息。因此可以去降维到9而不损失信息。

后续可以测试主成分为1-9的效果。

# 测试直接对step1的unnormalized数据进行PCA降维
这里先忽略离散特征应和连续特征区分的“正确手段”。

In [26]:
dirPath = "data preprocessed\\unnormalized - use all data - use PCA"
if not os.path.exists(dirPath):
    os.makesdir(dirPath)

先测试主成分为1-6的效果。

In [29]:
for k in tnrange(1, 6+1):
    fileName = '\\all-unnormalized-data-PCA=' + str(k) + '.csv'
    lowDDataMat, _ = PCA(mat(trainSet), k)
    DF(lowDDataMat).to_csv(dirPath + fileName, index=False, header=False)